In [4]:
import sys
import os
import hashlib
import struct
import subprocess
import collections
import tensorflow as tf
from tensorflow.core.example import example_pb2
import json
import io
import shutil
import subprocess

In [5]:
# generate mapping.txt and run the command manually
def tokenize_stories(stories_dir, tokenized_stories_dir):
    """Maps a whole directory of .story files to a tokenized version using Stanford CoreNLP Tokenizer"""
    print "Preparing to tokenize %s to %s..." % (stories_dir, tokenized_stories_dir)
    stories = os.listdir(stories_dir)
    # make IO list file
    print "Making list of files to tokenize..."
    if not os.path.isdir(separated_dir):
        os.mkdir(separated_dir)
    with open("mapping.txt", "w") as f:
        for s in stories:
            f.write("%s \t %s\n" % (os.path.join(stories_dir, s), os.path.join(tokenized_stories_dir, s)))
    # call system command
    #os.system("java edu.stanford.nlp.process.PTBTokenizer -ioFileList -preserveLines /home/xw0078/jupyter_notebooks/ETD/mapping.txt") 
    mapping_path = "/home/xw0078/jupyter_notebooks/ETD/mapping.txt"
    os.environ['CLASSPATH'] = "/home/xw0078/my_programs/lib/stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar"
    #command_1 = "export CLASSPATH=/home/xw0078/my_programs/lib/stanford-corenlp-full-2018-10-05/stanford-corenlp-3.9.2.jar"
    command_2 = "java edu.stanford.nlp.process.PTBTokenizer -ioFileList -preserveLines /home/xw0078/jupyter_notebooks/ETD/mapping.txt"
    #subprocess.check_output(command_1,shell=True)
    subprocess.check_output(command_2,shell=True)
    os.remove("mapping.txt")
    print "Tokenization Finished"

In [6]:
# load and separate Json file to abstract and body text
def separate_json(input_path, separated_dir):
    # Open the file with read only permit
    jsonFile = io.open(input_path,'r', encoding='utf8')
    # use readline() to read the first line 
    line = jsonFile.readline()
    # use the read line to read further.
    # If the file is not empty keep reading one line
    # at a time, till the file is empty
    ID = 0
    flag = 0
    empty_count = 0
    while line:
        record = json.loads(line)
        abstract =  record['introduction']
        body = record['textbody']
        if len(abstract) < 50:
            line = jsonFile.readline()
            continue
        if len(body) < 100:
            line = jsonFile.readline()
            continue
        #write abstract
        if not os.path.isdir(separated_dir):
            os.mkdir(separated_dir)
        f = open(separated_dir+str(ID)+".abs","wb")
        f.write(abstract.encode("utf-8",'ignore'))
        f.close()
        #write bodytext
        f = open(separated_dir+str(ID)+".body","wb")
        f.write(body.encode("utf-8",'ignore'))
        f.close()

        #next line
        ID = ID+1
        line = jsonFile.readline()
    jsonFile.close()

In [7]:
input_path = "/mnt/6t/wiki/processed/ETD_related_abstract_v4.json"
separated_dir = "/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/separated_files/"

shutil.rmtree(separated_dir)
os.mkdir(separated_dir)
separate_json(input_path,separated_dir)

In [8]:
input_dir = "/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/separated_files/"
tokenized_dir = "/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/tokenized_separated_files/"

shutil.rmtree(tokenized_dir)
os.mkdir(tokenized_dir)
tokenize_stories(input_dir,tokenized_dir)

Preparing to tokenize /mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/separated_files/ to /mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/tokenized_separated_files/...
Making list of files to tokenize...
Tokenization Finished


In [9]:
def get_art_abs(input_dir,ID):
    with open(input_dir+ID+".abs", 'r') as absFile:
        abstract = absFile.read()
    with open(input_dir+ID+".body", 'r') as bodyFile:
        body = bodyFile.read()
    return body,abstract

In [10]:
# Write to bin file
dm_single_close_quote = u'\u2019' # unicode
dm_double_close_quote = u'\u201d'
END_TOKENS = ['.', '!', '?', '...', "'", "`", '"', dm_single_close_quote, dm_double_close_quote, ")"] # acceptable ways to end a sentence
VOCAB_SIZE = 200000
# We use these to separate the summary sentences in the .bin datafiles
SENTENCE_START = '<s>'
SENTENCE_END = '</s>'
def write_to_bin(input_dir, out_file, makevocab=False):
    """Reads the tokenized .story files corresponding to the urls listed in the url_file and writes them to a out_file."""
    if makevocab:
        vocab_counter = collections.Counter()
    ID = 0
    with open(out_file, 'wb') as writer:
        # Get the strings to write to .bin file
        num_files = len(os.listdir(input_dir))/2-1
        for ID in range(0,num_files):
            
            article, abstract = get_art_abs(input_dir,str(ID))

            # Write to tf.Example
            tf_example = example_pb2.Example()
            tf_example.features.feature['article'].bytes_list.value.extend([article])
            tf_example.features.feature['abstract'].bytes_list.value.extend([abstract])
            tf_example_str = tf_example.SerializeToString()
            str_len = len(tf_example_str)
            if not str_len:
                print "MyError-0"
            writer.write(struct.pack('q', str_len))
            writer.write(struct.pack('%ds' % str_len, tf_example_str))

            # Write the vocab to file, if applicable
            if makevocab:
                art_tokens = article.split(' ')
                abs_tokens = abstract.split(' ')
                abs_tokens = [t for t in abs_tokens if t not in [SENTENCE_START, SENTENCE_END]] # remove these tags from vocab
                tokens = art_tokens + abs_tokens
                tokens = [t.strip() for t in tokens] # strip
                tokens = [t for t in tokens if t!=""] # remove empty
                vocab_counter.update(tokens)

    print "Finished writing file %s\n" % out_file

    # write vocab to file
    if makevocab:
        print "Writing vocab file..."
        with open(os.path.join("/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/finished_files/", "vocab_wikiETD"), 'w') as writer:
            for word, count in vocab_counter.most_common(VOCAB_SIZE):
                writer.write(word + ' ' + str(count) + '\n')
        print "Finished writing vocab file"

In [11]:
input_dir = "/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/tokenized_separated_files/"
output_dir = "/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/finished_files/train.bin"
write_to_bin(input_dir,output_dir,makevocab=True)

Finished writing file /mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/finished_files/train.bin

Writing vocab file...
Finished writing vocab file


In [71]:
chunks_dir = "/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/finished_files/chunked/"
CHUNK_SIZE = 1000 # num examples per chunk, for the chunked data
def chunk_file(set_name):
    in_file = '/mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/finished_files/%s.bin' % set_name
    reader = open(in_file, "rb")
    chunk = 0
    finished = False
    while not finished:
        chunk_fname = os.path.join(chunks_dir, '%s_wikiETD_%03d.bin' % (set_name, chunk)) # new chunk
        with open(chunk_fname, 'wb') as writer:
            for _ in range(CHUNK_SIZE):
                len_bytes = reader.read(8)
                if not len_bytes:
                    finished = True
                    break
                str_len = struct.unpack('q', len_bytes)[0]
                example_str = struct.unpack('%ds' % str_len, reader.read(str_len))[0]
                writer.write(struct.pack('q', str_len))
                writer.write(struct.pack('%ds' % str_len, example_str))
            chunk += 1
        
def chunk_all():
  # Make a dir to hold the chunks
    if not os.path.isdir(chunks_dir):
        os.mkdir(chunks_dir)
  # Chunk the data
    for set_name in ['train']:
        print "Splitting %s data into chunks..." % set_name
        chunk_file(set_name)
    print "Saved chunked data in %s" % chunks_dir

In [72]:
chunk_all()

Splitting train data into chunks...
Saved chunked data in /mnt/6t/DeepLearningPreTrainModels/wiki_etd_data/finished_files/chunked/
